In [62]:
# import the required Python scientific libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.io as pio
pio.renderers.default = 'iframe'
from types import SimpleNamespace
from plotly import graph_objects as go

## A python notebook to calculate lithosphere strength envelopes (modified from Marcos A. Lopez-Sanchez)
##### https://github.com/marcoalopez/strength_envelopes

In [63]:
def power_law_creep(strain_rate, n, A, E, T):
    """ Return the necessary differential stress (Tresca criterion) in
    MPa for permanently deforming a polycrystalline material at a given
    strain rate and environmental conditions (only considering T). This
    is a simplified flow law that ignores the effect of any variable other
    than temperature and material-related constants.

    Parameters (scalars and/or arrays, all positive values)
    ----------
    strain_rate : strain rate [s**-1]........scalar or array
    n : stress exponent......................scalar
    A : material constant [MPa**-n s**-1]....scalar
    E : activation energy [J mol**-1]........scalar
    T : absolute temperature [K].............scalar or array

    Assumptions
    -----------
    - Steady-state creep
    - Low to moderate stress regime (< 200 MPa)
    - The effect of pressure, grain size, water content, and
    partial melt is ignored
    """
    R = 8.3144626  # set universal gas constant [J mol**-1 K**-1]
    
    return (strain_rate * np.exp(E / (R * T)) / A)**(1 / n)

In [64]:
def calc_average_density(depths, moho, Lab, crust_densities, crust_layers, mantle_density):
    """ Return the corresponding average density at specific depths.

    Parameters
    ----------
    depths : array-like
        a list with the depths
    moho : positive scalar
        the depth of the moho discontinuity
    Lab : positive scalar
        the depth of the lithosphere-asthenosphere boundary
    crust_densities : tuple with three values
        the average density of the different crust layers
    crust_layers : tuple with two values
        depth of the base with respect to the depth of the moho
    mantle_density : positive scalar
        the average density of the lithospheric mantle

    Assumptions
    -----------
    - Assumes a three-layer crust and a single-layer lithospheric mantle
    - The uppermost layer start at depth 0
    """

    # preallocate
    densities = np.zeros(len(depths))
    step = depths[1] - depths[0]

    for index, depth in enumerate(depths):

        if depth <= crust_layers[0] * moho:  # upper crust
            avg_density = crust_densities[0]
        elif depth <= crust_layers[1] * moho:  # middle crust
            avg_density = densities[index - 1] * ((depth - step) / depth) + crust_densities[1] * (step / depth)
        elif depth <= moho:  # lower crust
            avg_density = densities[index - 1] * ((depth - step) / depth) + crust_densities[2] * (step / depth)
        elif depth <= Lab:  # lithospheric mantle
            avg_density = densities[index - 1] * ((depth - step) / depth) + mantle_density * (step / depth)

        densities[index] = avg_density

    return densities

Curated lits of experimentally derived values defining quartz and olivine flow laws from Marco A. Lopez (https://github.com/marcoalopez/strength_envelopes): 

In [65]:
def quartz(flow_law=None):
    """Curated list of experimentally derived values defining quartz flow laws.

    flow_law : string or None
        the flow law parameters to use, either...

    Returns
    -------
    A SimpleNamespace object containing the stress exponent (n),
    the activation energy (Q) [J mol**-1], the material
    constant (A) [MPa**-n s**-1], and the reference.
    """

    if flow_law is None:
        print('Available flow laws:')
        print("'HTD' from Hirth et al. (2004)")
        print("'LP_wet' from Luan and Paterson (1992)")
        print("'GT_wet' from Gleason and Tullis (1995)")
        print("'RD_wet' from Rybacki and Dresen (2000)")
        print("'HK_wet' from Holyoke and Kronenberg (2010)")
        print("'RB_wet' from Rutter and Brodie (2004)")
        return None

    elif flow_law == 'HTD':  # from Hirth et al. (2001)
        n = 4.0          # stress exponent
        E = 135000       # activation energy [J mol**-1]
        A = 10**(-11.2)  # material parameter [MPa**-n s**-1]
        ref = 'Hirth et al. (2004)'

    elif flow_law == 'LP_wet':  # from Luan and Paterson (1992)
        n = 4.0
        E = 152000
        A = 10**(-7.2)
        ref = 'Luan and Paterson (1992)'

    elif flow_law == 'GT_wet':  # from Gleason and Tullis (1995). Wet quartzite.
        n = 4.0
        E = 223000
        A = 1.1e-4
        ref = 'Gleason and Tullis (1995)'
    
    elif flow_law == 'RD_wet':  # Rybacki and Dresen (2000), wet anorthite for mafic crust.
        n = 3
        E = 356000
        A = 10**2.6
        ref = 'Rybacki and Dresen (2000)'

    elif flow_law == 'HK_wet':  # Holyoke and Kronenberg (2010), based on Gleason and Tullis (1995) data
        n = 4.0
        E = 223000
        A = 5.1e-4
        ref = 'Holyoke and Kronenberg (2010)'
        
    elif flow_law == 'RB_wet':  # from Rutter and Brodie (2004). Wet quartzite, minor grain boundary sliding inferred.
        n = 2.97
        E = 242000
        A = 10**(-4.93)
        ref = 'Rutter and Brodie (2004)'

    else:
        raise ValueError("Quartz flow law name misspelled. Use 'HTD', 'LP_wet', 'GT_wet', 'RD_wet', 'HK_wet' or 'RB_wet'")

    return SimpleNamespace(n=n, E=E, A=A, ref=ref)


def olivine(flow_law=None):
    """Curated list of experimentally derived values defining olivine flow laws.

    flow_law : string or None
        the flow law default parameters

    Returns
    -------
    A SimpleNamespace object containing the stress exponent (n),
    the activation energy (Q) [J mol**-1], the material constant (A)
    [MPa**-n s**-1], the activation volume per mol (V) [m**3 mol**-1],
    the water fugacity exponent (r), and the reference.
    """

    if flow_law is None:
        print('Available flow laws:')
        print("'KW_wet' from Karato and Wu (1993)")
        print("'HK_wet' from Hirth and Kohlstedt (2003)")
        print("'HK_dry' from Hirth and Kohlstedt (2003)")
        print("'KJ_wet' from Karato and Jung (2003)")
        print("'KJ_dry' from Karato and Jung (2003)")
        print("'ZK_dry' (dry peridotite) from Zimmerman and Kohlstedt (2004)")
        print("'Faul_dry' from Faul et al. (2011)")
        print("'Ohuchi' from Ohuchi et al. (2015)")
        return None
    
    elif flow_law == 'KW_wet':  # from Karato and Wu (1993). Wet olivine
        n = 3          # stress exponent
        E = 430000     # activation energy [J mol**-1]
        A = 10**(3.2)  # material paramenter [MPa**-n s**-1]
        V = 1.5e-5     # activation volume per mol [m**3 mol**-1]
        r = np.nan     # water fugacity exponent
        ref = 'Karato and Wu (1993)'

    elif flow_law == 'HK_wet':  # from Hirth and Kohlstedt (2003). Wet Olivine
        n = 3.5        # stress exponent
        E = 520000     # activation energy [J mol**-1]
        A = 10**(3.2)  # material parameter [MPa**-n s**-1]
        V = 2.2e-05    # activation volume per mol [m**3 mol**-1]
        r = np.nan     # water fugacity exponent
        ref = 'Hirth and Kohlstedt (2003)'

    elif flow_law == 'HK_dry':  # from Hirth and Kohlstedt (2003). Dry Olivine
        n = 3.5
        E = 530000
        A = 10**(5.0)
        V = 1.8e-05
        r = np.nan  # not provided
        ref = 'Hirth and Kohlstedt (2003)'

    elif flow_law == 'KJ_wet':  # from Karato and Jung (2003). Wet olivine
        n = 3.0
        E = 470000
        A = 10**(2.9)
        V = 2.4e-05
        r = np.nan  # not provided
        ref = 'Karato and Jung (2003)'

    elif flow_law == 'KJ_dry':  # from Karato and Jung (2003). Dry olivine
        n = 3.0
        E = 510000
        A = 10**(6.1)
        V = 1.4e-05
        r = np.nan  # not provided
        ref = 'Karato and Jung (2003)'

    elif flow_law == 'ZK_dry':  # from Zimmerman and Kohlstedt (2004). Dry peridotite
        n = 4.3
        E = 550000
        A = 10**(4.8)
        V = np.nan  # activation volume per mol not provided!
        r = np.nan  # not provided
        ref = 'Zimmerman and Kohlstedt (2004)'

    elif flow_law == 'Faul_dry':  # from Faul et al. (2011). Dry olivine
        n = 8.2
        E = 682000
        A = 0.3
        V = np.nan  # activation volume per mol not provided!
        r = np.nan  # not provided
        ref = 'Faul et al. (2011)'

    elif flow_law == 'Ohuchi':  # from Ohuchi et al. (2015)
        n = 3.0
        E = 423000
        A = 10**(-4.89)
        V = 17.6e-06
        r = 1.25
        ref = 'Ohuchi et al. (2015)'

    else:
        raise ValueError("Olivine flow law name misspelled. Use 'KW_wet', 'HK_wet', 'HK_dry', 'KJ_wet', 'KJ_dry', 'ZK_dry', 'Faul_dry', or 'Ohuchi'")

    return SimpleNamespace(n=n, E=E, A=A, V=V, r=r, ref=ref)


def olivine_Idrissi(R, T, stress):
    """Semi-empirical olivine flow law for the uppermost mantle based on Idrissi
    et al. (2016). It solves the equation:

    ss = 1e6 * exp{(-566e3 / (R * T)) * [1 - sqrt(stress / 3.8)]**2}

    Parameters
    ----------
    R : scalar
        universal gas constant

    geotherm : scalar
        the temperature in K

    stress : scalar or numpy array_like
        the stress in MPa

    Returns
    -------
    the strain rate in s**-1
    """
    # convert from MPa to GPa
    stress = stress / 1000

    return 1e6 * np.exp((-556e3 / (R * T)) * (1 - np.sqrt(stress / 3.8))**2)

Thermal functions used for the calculation and obtained from Marcos A. Lopez (https://github.com/marcoalopez/strength_envelopes):

In [66]:
def turcotte_schubert_model(depths, thermal):
    """ Apply the equation (model) of Turcotte and Schubert (1982) (ts) to estimate
    a steady-state geotherm (i.e. the T at a given depth)

    Parameters (arrays or scalar with positive values)
    ----------
    depths : an array with the depths
    
    thermal : a tuple of dim 4 (T0, Jq, A, K)
        T0 is the temperature at the min depth [K]
        Jq is the average heat flux [mW m**-2]
        A is the average heat productivity [microW m**-3]
        K is the coefficient of thermal conductivity [W m**-1 K**-1]

    Assumptions
    -----------
    - the temperature only vary as a function of depth (not in time, i.e. steady-state geotherm).
    - Heat is transferred by conduction (as in the lithosphere).
    - The temperature gradient depends on heat conduction plus the heat produced due to radioactive decay.
    - Radioactive heat production are independent of depth.

    Returns
    -------
    The temperature in K, a float
    """
    # extract the different thermal parameters
    T0, Jq, A, K = thermal
    
    # get the min depth in the model [km]
    z0 = depths[0]
    
    return T0 + Jq / K * (depths - z0) - A / (2 * K) * (depths - z0)**2


def thermal_conductivity(T, K_0):
    """ Estimate the temperature-dependent thermal conductivity based on
    the model of Druham et al. (1987)

    Parameters
    ----------
    T : absolute temperature at Earth surface [K]
    K_0 : conductivity at the surface [mW m**-2] (for T=273 K)

    Assumptions
    -----------
  
    - only reliable for temperatures below 1000 K (at higher values
    the radioactive component must be included)

    Returns
    -------
    The thermal conductivity in W m**-1 K**-1, a floating point number
    """

    second_term = 618.241 / T
    third_term = K_0 * ((255.576 / T) - 0.30247)

    return 2.26 - second_term + third_term

Function to calculate the corresponding differential stress in MPa for a specific depth based on the Anderson theory of faulting (Anderson, 1905) and the Coulomb-Navier's law of friction

In [67]:
def Anderson_fault(fault_type, depths_fric, densities_fric, mu, C0, lamb, g):
    """ Returns the corresponding differential stress in MPa for a specific depth
    based on the Anderson theory of faulting (Anderson, 1905) and the Coulomb–
    Navier’s law of friction.

    Parameters
    ----------
    fault_type : string
        the type of fault, either 'inverse', 'normal' or 'strike-slip'
    depths : array-like
        an array-like with the depths [km]
    densities : array-like
        the corresponding average density [kg m**-3]
    mu : scalar between 0 an 1, optional
        Coefficient of friction. Default value 0.73; this is the Rutter and Glover
        (2012) coefficient recalculated from Byerlee (1978) data
    C0 : positive scalar
        internal cohesion of the rock [MPa]. Mostly negligible in nature. Default = 0.0
        This parameter can be used as the frictional cohesive strenght as well.
    lamb : scalar between 0 and 1, optional
        Hubbert-Rubbey coefficient of fluid pressure. Zero is for dry conditions.
        Default = 0.36
    g : scalar
        average gravitational acceleration [m s**-2]
    """

    depths_fric = 1000 * depths_fric  # convert km to m
        
    if fault_type == 'inverse':
        diff_stress = (2 * (C0 + mu * densities_fric * g * depths_fric * (1 - lamb))) / (np.sqrt(mu**2 + 1) - mu)

    elif fault_type == 'strike-slip':
        diff_stress = (2 * (C0 + mu * densities_fric * g * depths_fric * (1 - lamb))) / (np.sqrt(mu**2 + 1))

    elif fault_type == 'normal':
        diff_stress = (- 2 * (C0 - mu * densities_fric * g * depths_fric * (1 - lamb))) / (np.sqrt(mu**2 + 1) + mu)

    return diff_stress / 10**6

Function to estimate frictional strength slopes in the depth vs. differential stress space (modified from Marcos A. Lopez, https://github.com/marcoalopez/strength_envelopes)

In [68]:
def fric_strength(dataset_1, max_depth, fault_type, mu, lamb, C0, annot, **kwargs):
    
    depths_fric = depths[depths <= max_depth]
    densities_fric = densities[depths <= max_depth]

    # Compute differential stress values depending on the type of fault
    if fault_type == 'inverse' or 'normal' or 'strike-slip':
            diffs = Anderson_fault(fault_type, depths_fric, densities_fric, mu, C0, lamb, g)
    else:
        raise ValueError("Faul type misspelled. Please use 'inverse', 'normal' or 'strike-slip'.")

    print('')
    print('fault type:', fault_type)
    print('Coefficient of friction =', mu)
    print('Coefficient of fluid pressure =', lamb)
    print('Internal cohesion (or frictional cohesive strength) =', C0)
    print(' ')

    return diffs

## Neoproterozoic lithospheric model set up:

In [69]:
moho = 48      # Neoproterozoic crust thickness [km], average from Christensen and Mooney (1995) and Svartman Dias et al. (2015)
Lab = 140        # Neoproterozoic lithosphere thickness [km] from Artemieva and Mooney (2001)
T0 = 7.5 + 273.15         # surface temperature in K (taken from the KTB superdeep borehole)
crust_densities = (2750, 2800, 2900)      # Kg cm*-3
crust_layers = (0.5, 0.7)      # depth of the base of the crustal layer with respect to the depth of the moho
mantle_density = 3250      # Kg cm**-3

#Set the average heat parameters for the crust.
#the thermal parameters are stored within two different objects called heat_crust and heat_mantle:
#Jq : average heat flux [mW m**-2]
#A : average heat productivity [microW m**-3]
#K : coefficient of thermal conductivity [W m**-1 K**-1]

heat_crust = SimpleNamespace(Jq=70, A=1, K=3)      # crustal and mantle thermal parameters for Neoproterozoic terranes. Average heat flux (Jq) obtained from Artemieva (2011), table 4.29. Average heat productivity (A) value obtained from Artemieva (2011), table 4.8. Coefficient of thermal conductivity (K) obtained from Artemiva (2011), table 4.3. 
heat_mantle = SimpleNamespace(Jq=40, A=0.3, K=4)

g = 9.80665      # average gravitational acceleration [m s**-2]

print(' ')
print('THE MODELLED NEOPROTEROZOIC LITHOSPHERE COMPRISES:')
print('a {val} [km] thick Upper Crust' .format(val=round(crust_layers[0]*moho)))
print('a {val} [km] thick Middle Crust' .format(val=round(crust_layers[1]*moho-crust_layers[0]*moho)))
print('a {val} [km] thick Lower Crust' .format(val=round(moho-crust_layers[1]*moho)))
print('and a {val} [km] thick lithospheric mantle' .format(val=round(Lab-moho)))

 
THE MODELLED NEOPROTEROZOIC LITHOSPHERE COMPRISES:
a 24 [km] thick Upper Crust
a 10 [km] thick Middle Crust
a 14 [km] thick Lower Crust
and a 92 [km] thick lithospheric mantle


In [70]:
 # Estimate a mesh density so that there is a measure every 10 m
mesh_density = Lab*100

 # Get a list of depths [km] and corresponding average density [kg/m**3] and pressures (MPa)
depths = np.linspace(0, Lab, mesh_density)
densities = calc_average_density(depths, moho, Lab, crust_densities, crust_layers, mantle_density)
pressures = densities * g * depths / 1e3      # /1e3 to obtain MPa
dataset_1 = pd.DataFrame({'depths_km': depths,
                          'mean_density': densities,
                         'Pressure_MPa': pressures})      #dataframe containing the different parameters of the lithospheric model 
dataset_1.round(2)

,depths_km,mean_density,Pressure_MPa
0,0.00,2750.00,0.00
1,0.01,2750.00,0.27
2,0.02,2750.00,0.54
3,0.03,2750.00,0.81
4,0.04,2750.00,1.08
...,...,...,...
13995,139.96,3097.41,4251.31
13996,139.97,3097.42,4251.63
13997,139.98,3097.43,4251.95
13998,139.99,3097.44,4252.27


Calculating the frictional strength slopes in the depth versus differential stress space for different coefficients of friction (mu):

In [71]:
diffs_10 = fric_strength(dataset_1,max_depth=140,fault_type='normal',mu=0.1,lamb=0.1,C0=0.0,annot='mu')
diffs_20 = fric_strength(dataset_1,max_depth=140,fault_type='normal',mu=0.2,lamb=0.1,C0=0.0,annot='mu')
diffs_30 = fric_strength(dataset_1,max_depth=140,fault_type='normal',mu=0.3,lamb=0.1,C0=0.0,annot='mu')
diffs_73 = fric_strength(dataset_1,max_depth=140,fault_type='normal',mu=0.73,lamb=0.1,C0=0.0,annot='mu')
dataset_1['diffs_10_MPa'] = diffs_10
dataset_1['diffs_20_MPa'] = diffs_20
dataset_1['diffs_30_MPa'] = diffs_30
dataset_1['diffs_73_MPa'] = diffs_73
dataset_1


fault type: normal
Coefficient of friction = 0.1
Coefficient of fluid pressure = 0.1
Internal cohesion (or frictional cohesive strength) = 0.0
 

fault type: normal
Coefficient of friction = 0.2
Coefficient of fluid pressure = 0.1
Internal cohesion (or frictional cohesive strength) = 0.0
 

fault type: normal
Coefficient of friction = 0.3
Coefficient of fluid pressure = 0.1
Internal cohesion (or frictional cohesive strength) = 0.0
 

fault type: normal
Coefficient of friction = 0.73
Coefficient of fluid pressure = 0.1
Internal cohesion (or frictional cohesive strength) = 0.0
 


,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa
0,0.000000,2750.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000
1,0.010001,2750.000000,0.269702,0.043934,0.079597,0.108360,0.180066
2,0.020001,2750.000000,0.539404,0.087868,0.159194,0.216720,0.360132
3,0.030002,2750.000000,0.809106,0.131802,0.238791,0.325080,0.540198
4,0.040003,2750.000000,1.078809,0.175735,0.318388,0.433440,0.720264
...,...,...,...,...,...,...,...
13995,139.959997,3097.409789,4251.314824,692.529667,1254.688014,1708.078609,2838.381211
13996,139.969998,3097.420692,4251.633563,692.581589,1254.782084,1708.206671,2838.594016
13997,139.979999,3097.431592,4251.952302,692.633511,1254.876153,1708.334733,2838.806822
13998,139.989999,3097.442492,4252.271041,692.685432,1254.970222,1708.462795,2839.019627


In [72]:
# Estimate stable geotherm for the crust: call the "turcotte_schubert_model" and pass the different arguments in order
T_crust = turcotte_schubert_model(depths[depths <= moho],thermal=(T0, heat_crust.Jq, heat_crust.A, heat_crust.K))

# Estimate average thermal gradient of the crust (K Km**-1)
Tg_crust = ((heat_crust.Jq / 1000) / heat_crust.K)*1000


#Retrieving Temp values at Moho from dataset_1 and converting to degrees C
T_at_moho_index = int(np.argwhere(depths <= moho)[-1])
T_at_moho = T_crust[T_at_moho_index]

print(' ')
print('ACCORDING TO THE MODEL:')
print('The temperature at the base of the Moho is {val} [K]' .format(val=round(T_at_moho, 1)),('({val} [deg C])'.format(val=round(T_at_moho-273.15, 1))))

 
ACCORDING TO THE MODEL:
The temperature at the base of the Moho is 1016.6 [K] (743.5 [deg C])


In [73]:
# Estimate stable geotherm: call the "turcotte_schubert_model" and pass the different arguments in order for the lithospheric mantle
T_mantle = turcotte_schubert_model(depths[depths > moho],thermal=(T_at_moho, heat_mantle.Jq, heat_mantle.A, heat_mantle.K))

# Estimate average thermal gradient of the mantle (K Km**-1)
Tg_mantle = ((heat_mantle.Jq / 1000) / heat_mantle.K)*1000

# Concatenating T_crust and T_mantle values
T_values = np.hstack((T_crust, T_mantle))

# Adding temp values in Kelvin (T_values) of the whole lithopshere to dataset_1
dataset_1['T_values_K'] = T_values

print(' ')
print('ACCORDING TO THE MODEL:')
print('The temperature at the lithosphere-asthenosphere boundary is {val} [deg C]' .format(val=round(T_values[-1] - 273.15, 1)))
print('The average temperature gradient in the crust is {val} [K Km**-1]' .format(val=round(Tg_crust, 2)))
print('The average temperature gradient in the lithospheric mantle is {val} [K Km**-1]' .format(val=round(Tg_mantle, 2)))
dataset_1.round(2)

 
ACCORDING TO THE MODEL:
The temperature at the lithosphere-asthenosphere boundary is 1346.0 [deg C]
The average temperature gradient in the crust is 23.33 [K Km**-1]
The average temperature gradient in the lithospheric mantle is 10.0 [K Km**-1]


,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K
0,0.00,2750.00,0.00,-0.00,-0.00,-0.00,-0.00,280.65
1,0.01,2750.00,0.27,0.04,0.08,0.11,0.18,280.88
2,0.02,2750.00,0.54,0.09,0.16,0.22,0.36,281.12
3,0.03,2750.00,0.81,0.13,0.24,0.33,0.54,281.35
4,0.04,2750.00,1.08,0.18,0.32,0.43,0.72,281.58
...,...,...,...,...,...,...,...,...
13995,139.96,3097.41,4251.31,692.53,1254.69,1708.08,2838.38,1619.07
13996,139.97,3097.42,4251.63,692.58,1254.78,1708.21,2838.59,1619.10
13997,139.98,3097.43,4251.95,692.63,1254.88,1708.33,2838.81,1619.13
13998,139.99,3097.44,4252.27,692.69,1254.97,1708.46,2839.02,1619.16


In [74]:
# Exporting dataset_1 to a .csv file. Uncomment to write output to file.
#dataset_1.to_csv('Neoproterozoic_lithosphere_model.csv', index=True)

In [75]:
dataset_1_crust = dataset_1.iloc[0:4800, :].copy()
dataset_1_crust

,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K
0,0.000000,2750.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,280.650000
1,0.010001,2750.000000,0.269702,0.043934,0.079597,0.108360,0.180066,280.883333
2,0.020001,2750.000000,0.539404,0.087868,0.159194,0.216720,0.360132,281.116633
3,0.030002,2750.000000,0.809106,0.131802,0.238791,0.325080,0.540198,281.349900
4,0.040003,2750.000000,1.078809,0.175735,0.318388,0.433440,0.720264,281.583133
...,...,...,...,...,...,...,...,...
4795,47.953425,2804.932221,1319.054320,214.870996,389.291717,529.965096,880.663784,1016.308090
4796,47.963426,2804.952043,1319.338733,214.917326,389.375655,530.079366,880.853671,1016.381567
4797,47.973427,2804.971857,1319.623146,214.963656,389.459594,530.193637,881.043559,1016.455011
4798,47.983427,2804.991663,1319.907559,215.009986,389.543533,530.307907,881.233447,1016.528422


In [76]:
dataset_1_mantle = dataset_1.iloc[4801:14000, :].copy()
dataset_1_mantle

,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K
4801,48.013430,2805.196834,1320.829450,215.160160,389.815610,530.678302,881.848946,1016.701802
4802,48.023430,2805.289463,1321.148189,215.212082,389.909679,530.806363,882.061751,1016.801798
4803,48.033431,2805.382053,1321.466928,215.264004,390.003748,530.934425,882.274556,1016.901787
4804,48.043432,2805.474604,1321.785667,215.315926,390.097817,531.062487,882.487362,1017.001767
4805,48.053432,2805.567118,1322.104406,215.367848,390.191887,531.190549,882.700167,1017.101741
...,...,...,...,...,...,...,...,...
13995,139.959997,3097.409789,4251.314824,692.529667,1254.688014,1708.078609,2838.381211,1619.067090
13996,139.969998,3097.420692,4251.633563,692.581589,1254.782084,1708.206671,2838.594016,1619.098121
13997,139.979999,3097.431592,4251.952302,692.633511,1254.876153,1708.334733,2838.806822,1619.129145
13998,139.989999,3097.442492,4252.271041,692.685432,1254.970222,1708.462795,2839.019627,1619.160161


In [77]:
quartz()

Available flow laws:
'HTD' from Hirth et al. (2004)
'LP_wet' from Luan and Paterson (1992)
'GT_wet' from Gleason and Tullis (1995)
'RD_wet' from Rybacki and Dresen (2000)
'HK_wet' from Holyoke and Kronenberg (2010)
'RB_wet' from Rutter and Brodie (2004)


In [78]:
Hirth = quartz('HTD')
Luan = quartz('LP_wet')
Gleason = quartz('GT_wet')
Rybacki = quartz('RD_wet')
Holyoke = quartz('HK_wet')
Rutter = quartz('RB_wet')

# check that we obtained the three values
print(' ')
print('Checking the three values of each power law considered for the modeling:')
print(Hirth)
print(Luan)
print(Gleason)
print(Rybacki)
print(Holyoke)
print(Rutter)
print(' ')

 
Checking the three values of each power law considered for the modeling:
namespace(A=6.309573444801943e-12, E=135000, n=4.0, ref='Hirth et al. (2004)')
namespace(A=6.30957344480193e-08, E=152000, n=4.0, ref='Luan and Paterson (1992)')
namespace(A=0.00011, E=223000, n=4.0, ref='Gleason and Tullis (1995)')
namespace(A=398.1071705534973, E=356000, n=3, ref='Rybacki and Dresen (2000)')
namespace(A=0.00051, E=223000, n=4.0, ref='Holyoke and Kronenberg (2010)')
namespace(A=1.1748975549395302e-05, E=242000, n=2.97, ref='Rutter and Brodie (2004)')
 


Estimate the differential stress at steady-state using 1.0e-15 $s^{-1}$ as the reference average shear strain rate in the ductile lithosphere.

In [79]:
dataset_1_crust['model_HTD'] = power_law_creep(strain_rate=1e-15,
                                       n=Hirth.n,
                                       A=Hirth.A,
                                       E=Hirth.E,
                                       T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust['model_LP_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Luan.n,
                                      A=Luan.A,
                                      E=Luan.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust['model_GT_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Gleason.n,
                                      A=Gleason.A,
                                      E=Gleason.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust['model_RD_wet'] = power_law_creep(strain_rate = 1e-15,
                                      n=Rybacki.n,
                                      A=Rybacki.A,
                                      E=Rybacki.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )                                                 

dataset_1_crust['model_HK_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Holyoke.n,
                                      A=Holyoke.A,
                                      E=Holyoke.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust['model_RB_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Rutter.n,
                                      A=Rutter.A,
                                      E=Rutter.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust.round(2)

,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K,model_HTD,model_LP_wet,model_GT_wet,model_RD_wet,model_HK_wet,model_RB_wet
0,0.00,2750.00,0.00,-0.00,-0.00,-0.00,-0.00,280.65,214503.72,132565.44,41271625.34,1.656412e+16,28125949.96,5.949533e+11
1,0.01,2750.00,0.27,0.04,0.08,0.11,0.18,280.88,211941.87,130784.16,40460579.47,1.587894e+16,27573235.22,5.779431e+11
2,0.02,2750.00,0.54,0.09,0.16,0.22,0.36,281.12,209415.15,129029.97,39666890.89,1.522325e+16,27032349.19,5.614486e+11
3,0.03,2750.00,0.81,0.13,0.24,0.33,0.54,281.35,206923.02,127302.40,38890159.13,1.459575e+16,26503018.98,5.454532e+11
4,0.04,2750.00,1.08,0.18,0.32,0.43,0.72,281.58,204464.96,125601.01,38129993.64,1.399518e+16,25984978.40,5.299412e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,47.95,2804.93,1319.05,214.87,389.29,529.97,880.66,1016.31,6.09,1.01,1.27,1.710000e+00,0.87,6.270000e+00
4796,47.96,2804.95,1319.34,214.92,389.38,530.08,880.85,1016.38,6.09,1.01,1.27,1.710000e+00,0.87,6.260000e+00
4797,47.97,2804.97,1319.62,214.96,389.46,530.19,881.04,1016.46,6.09,1.01,1.27,1.700000e+00,0.87,6.260000e+00
4798,47.98,2804.99,1319.91,215.01,389.54,530.31,881.23,1016.53,6.08,1.01,1.27,1.700000e+00,0.87,6.260000e+00


In [80]:
# Exporting dataset_1_crust to a .csv file. Uncomment to write output to file.
#dataset_1_crust.to_csv('Neoproterozoic_crust_model.csv', index=True)

In [81]:
olivine()

Available flow laws:
'KW_wet' from Karato and Wu (1993)
'HK_wet' from Hirth and Kohlstedt (2003)
'HK_dry' from Hirth and Kohlstedt (2003)
'KJ_wet' from Karato and Jung (2003)
'KJ_dry' from Karato and Jung (2003)
'ZK_dry' (dry peridotite) from Zimmerman and Kohlstedt (2004)
'Faul_dry' from Faul et al. (2011)
'Ohuchi' from Ohuchi et al. (2015)


In [82]:
Karato_0 = olivine('KW_wet')
Hirth_2 = olivine('HK_wet')
Hirth_3 = olivine('HK_dry')
Karato_1 = olivine('KJ_wet')
Karato_2 = olivine('KJ_dry')
Zimmerman = olivine('ZK_dry')
Faul = olivine('Faul_dry')
Ohuchi = olivine('Ohuchi')

# check that we obtained the three values
print(' ')
print('Checking the three values of each power law considered for the modeling:')
print(Karato_0)
print(Hirth_2) 
print(Hirth_3)
print(Karato_1)
print(Karato_2)
print(Zimmerman)
print(Faul)
print(Ohuchi)
print(' ')

 
Checking the three values of each power law considered for the modeling:
namespace(A=1584.893192461114, E=430000, V=1.5e-05, n=3, r=nan, ref='Karato and Wu (1993)')
namespace(A=1584.893192461114, E=520000, V=2.2e-05, n=3.5, r=nan, ref='Hirth and Kohlstedt (2003)')
namespace(A=100000.0, E=530000, V=1.8e-05, n=3.5, r=nan, ref='Hirth and Kohlstedt (2003)')
namespace(A=794.3282347242813, E=470000, V=2.4e-05, n=3.0, r=nan, ref='Karato and Jung (2003)')
namespace(A=1258925.411794166, E=510000, V=1.4e-05, n=3.0, r=nan, ref='Karato and Jung (2003)')
namespace(A=63095.7344480193, E=550000, V=nan, n=4.3, r=nan, ref='Zimmerman and Kohlstedt (2004)')
namespace(A=0.3, E=682000, V=nan, n=8.2, r=nan, ref='Faul et al. (2011)')
namespace(A=1.2882495516931348e-05, E=423000, V=1.76e-05, n=3.0, r=1.25, ref='Ohuchi et al. (2015)')
 


In [83]:
dataset_1_mantle['model_KW_wet'] = power_law_creep(strain_rate=1e-15,
                                                  n=Karato_0.n,
                                                  A=Karato_0.A,
                                                  E=Karato_0.E,
                                                  T=dataset_1_mantle['T_values_K'])

dataset_1_mantle['model_HK_wet'] = power_law_creep(strain_rate=1e-15,
                                       n=Hirth_2.n,
                                       A=Hirth_2.A,
                                       E=Hirth_2.E,
                                       T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_HK_dry'] = power_law_creep(strain_rate=1e-15,
                                      n=Hirth_3.n,
                                      A=Hirth_3.A,
                                      E=Hirth_3.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_KJ_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Karato_1.n,
                                      A=Karato_1.A,
                                      E=Karato_1.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_KJ_dry'] = power_law_creep(strain_rate=1e-15,
                                      n=Karato_2.n,
                                      A=Karato_2.A,
                                      E=Karato_2.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_ZK_dry'] = power_law_creep(strain_rate=1e-15,
                                      n=Zimmerman.n,
                                      A=Zimmerman.A,
                                      E=Zimmerman.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_Faul_dry'] = power_law_creep(strain_rate=1e-15,
                                      n=Faul.n,
                                      A=Faul.A,
                                      E=Faul.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_Ohuchi'] = power_law_creep(strain_rate=1e-15,
                                      n=Ohuchi.n,
                                      A=Ohuchi.A,
                                      E=Ohuchi.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle.round(2)

,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K,model_KW_wet,model_HK_wet,model_HK_dry,model_KJ_wet,model_KJ_dry,model_ZK_dry,model_Faul_dry,model_Ohuchi
4801,48.01,2805.20,1320.83,215.16,389.82,530.68,881.85,1016.70,19.82,270.98,116.26,120.83,50.18,92.61,321.69,7480.89
4802,48.02,2805.29,1321.15,215.21,389.91,530.81,882.06,1016.80,19.79,270.51,116.06,120.61,50.08,92.47,321.38,7468.63
4803,48.03,2805.38,1321.47,215.26,390.00,530.93,882.27,1016.90,19.76,270.05,115.85,120.39,49.98,92.33,321.07,7456.39
4804,48.04,2805.47,1321.79,215.32,390.10,531.06,882.49,1017.00,19.72,269.58,115.65,120.17,49.88,92.19,320.76,7444.18
4805,48.05,2805.57,1322.10,215.37,390.19,531.19,882.70,1017.10,19.69,269.12,115.45,119.95,49.78,92.06,320.45,7431.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13995,139.96,3097.41,4251.31,692.53,1254.69,1708.08,2838.38,1619.07,0.04,0.39,0.15,0.12,0.03,0.33,8.27,15.10
13996,139.97,3097.42,4251.63,692.58,1254.78,1708.21,2838.59,1619.10,0.04,0.39,0.15,0.12,0.03,0.33,8.27,15.09
13997,139.98,3097.43,4251.95,692.63,1254.88,1708.33,2838.81,1619.13,0.04,0.39,0.15,0.12,0.03,0.33,8.27,15.09
13998,139.99,3097.44,4252.27,692.69,1254.97,1708.46,2839.02,1619.16,0.04,0.39,0.15,0.12,0.03,0.33,8.27,15.09


In [84]:
# Exporting dataset_1_mantle to a .csv file. Uncomment to write output to file.
#dataset_1_mantle.to_csv('Neoproterozoic_mantle_model.csv', index=True)

### Strength envelope calculated for the Neoproterozoic lithosphere type:

In [85]:
fig = go.Figure()

# plot the limits of the lithosheric layers (i.e., Moho and LAB depths)
# crust area
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [moho, moho],
    mode='lines',
    line=dict(color='#F5F5DC'),
    fill = 'tozeroy',
    showlegend = False
))
# lithospheric mantle area
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [Lab, Lab],
    mode='lines', 
    line=dict(color='#70A459'),
    fill='tonexty',
    showlegend = False
))
# asthernosphere area
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [170, 170],
    mode= 'lines',
    line=dict(color='#fdbcb4'),
    fill='tonexty',
    showlegend = False
))

#moho line
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [Lab, Lab],
    mode='lines', 
    line=dict(color='white', width=1),
    showlegend = False
))
# lithosphere line
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [moho, moho],
    mode='lines',
    line=dict(color='white', width=1),
    showlegend = False
))
#TEXT
# moho text
fig.add_trace(go.Scatter(
    x = [100],
    y = [moho - moho/90],
    mode='text',
    text = 'Moho',
    name = 'moho text',
    showlegend = False
))
# lab text
fig.add_trace(go.Scatter(
    x = [310],
    y = [Lab - Lab/90],
    mode='text',
    text = 'Lithosphere base',
    name = 'litho text',
    showlegend = False
))
#asthenosphere text
fig.add_trace(go.Scatter(
    x = [300],
    y = [155 - 155/90],
    mode= 'text',
    text = 'Asthenosphere',
    name = 'Astheno text',
    showlegend = False
))
# plot frictional strength slopes in the depth vs differential stress space
fig.add_trace(go.Scatter(
    x = dataset_1['diffs_10_MPa']/1000,
    y = dataset_1['depths_km'],
    name = 'angle of internal friction = 10',
    mode='lines',
    xaxis='x2'
))
fig.add_trace(go.Scatter(
    x = dataset_1['diffs_20_MPa']/1000,
    y = dataset_1['depths_km'],
    name = 'angle of internal friction = 20',
    mode='lines',
    xaxis='x2'
))
fig.add_trace(go.Scatter(
    x = dataset_1['diffs_30_MPa']/1000,
    y = dataset_1['depths_km'],
    name = 'angle of internal friction = 30',
    mode='lines',
    xaxis='x2'
))

# plot the strength envelopes for the crust
fig.add_trace(go.Scatter(
    x = dataset_1_crust['model_GT_wet']/1000,
    y = dataset_1_crust['depths_km'],
    name = "Gleason and Tullis (1995)",
    mode = 'lines',
    xaxis = 'x2'
))
fig.add_trace(go.Scatter(
    x = dataset_1_crust['model_RD_wet']/1000,
    y = dataset_1_crust['depths_km'],
    name = "Rybacki and Dresen (2000)",
    mode = 'lines',
    xaxis = 'x2'
))

#plot geothermal gradient of the Neoproterozoic lithosphere
fig.add_trace(go.Scatter(
    x = dataset_1['T_values_K']-273,
    y = dataset_1['depths_km'],
    name = "Geothermal gradient",
    line=dict(color='red', width=3.5),
    mode = 'lines',
    xaxis = 'x'
))

# plot the strength envelopes for the mantle
author = 'model_Ohuchi'
legend = 'Ohuchi et al., (2015)'
fig.add_trace(go.Scatter(
    x = dataset_1_mantle['{}'.format(author)]/1000,
    y = dataset_1_mantle['depths_km'],
    name = legend, 
    mode='lines',
    xaxis = 'x2'
))


fig.update_layout(
    autosize = False,
    width = 800, #800
    height = 1200, #1200
    yaxis = dict(
        autorange = 'reversed',
        range = [0, 1100]
    ),
    xaxis =dict(
        title = 'Temperature [ºC]',
        range = [0, 2500]
    ),
    xaxis2 = dict(
        title = "Differential Stress [GPa]",
        overlaying = 'x',
        side = 'top',
        range = [0, 2]
    )
)
#fig.show(renderer="colab") #Uncomment to show graphical representation in Google Colab platform

In [86]:
# Exporting Neoproterozoic lithospheric strength envelope to pdf file. Uncomment to write output to file. 
#fig.write_image("Neoproterozoic_lithospheric_strength_envelope.pdf")

## Paleoproterozoic lithospheric model set up:

In [87]:
moho = 40      # Paleoproterozoic crust thickness [km], average from Christensen and Mooney (1995) and Svartman Dias et al. (2015)
Lab = 200        # Paleoproterozoic lithosphere thickness [km] from Artemieva and Mooney (2001)
T0 = 7.5 + 273.15         # surface temperature in K (taken from the KTB superdeep borehole)
crust_densities = (2700, 2750, 2850)      # Kg cm**-3
crust_layers = (0.55, 0.7)      # depth of the base of the crustal layer with respect to the depth of the moho
mantle_density = 3225      # Kg cm**-3

#Set the average heat parameters for the crust.
#the thermal parameters are stored within two different objects called heat_crust and heat_mantle:
#Jq : average heat flux [mW m**-2]
#A : average heat productivity [microW m**-3]
#K : coefficient of thermal conductivity [W m**-1 K**-1]

heat_crust = SimpleNamespace(Jq=50, A=0.9, K=2.75)      # crustal and mantle thermal parameters for Paleoproterozoic terranes. Average heat flux (Jq) obtained from Artemieva (2011), table 4.29. Average heat productivity (A) value obtained from Artemieva (2011), table 4.8. Coefficient of thermal conductivity (K) obtained from Artemiva (2011), table 4.3.
heat_mantle = SimpleNamespace(Jq=30, A=0.1, K=4)

g = 9.80665      # average gravitational acceleration [m s**-2]

print(' ')
print('THE MODELLED PALEOPROTEROZOIC LITHOSPHERE COMPRISES:')
print('a {val} [km] thick Upper Crust' .format(val=round(crust_layers[0]*moho)))
print('a {val} [km] thick Middle Crust' .format(val=round(crust_layers[1]*moho-crust_layers[0]*moho)))
print('a {val} [km] thick Lower Crust' .format(val=round(moho-crust_layers[1]*moho)))
print('and a {val} [km] thick lithospheric mantle' .format(val=round(Lab-moho)))

 
THE MODELLED PALEOPROTEROZOIC LITHOSPHERE COMPRISES:
a 22 [km] thick Upper Crust
a 6 [km] thick Middle Crust
a 12 [km] thick Lower Crust
and a 160 [km] thick lithospheric mantle


In [88]:
 # Estimate a mesh density so that there is a measure every 10 m
mesh_density = Lab*100

 # Get a list of depths [km] and corresponding average density [kg/m**3] and pressures (MPa)
depths = np.linspace(0, Lab, mesh_density)
densities = calc_average_density(depths, moho, Lab, crust_densities, crust_layers, mantle_density)
pressures = densities * g * depths / 1e3      # /1e3 to obtain MPa
dataset_1 = pd.DataFrame({'depths_km': depths,
                          'mean_density': densities,
                         'Pressure_MPa': pressures})      #dataframe containing the different parameters of the lithospheric model 
dataset_1.round(2)

,depths_km,mean_density,Pressure_MPa
0,0.00,2700.00,0.00
1,0.01,2700.00,0.26
2,0.02,2700.00,0.53
3,0.03,2700.00,0.79
4,0.04,2700.00,1.06
...,...,...,...
19995,199.96,3130.50,6138.72
19996,199.97,3130.51,6139.04
19997,199.98,3130.51,6139.35
19998,199.99,3130.52,6139.67


Calculating the frictional strength slopes in the depth versus differential stress space for different coefficients of friction (mu):

In [89]:
diffs_10 = fric_strength(dataset_1,max_depth=200,fault_type='normal',mu=0.1,lamb=0.1,C0=0.0,annot='mu')
diffs_20 = fric_strength(dataset_1,max_depth=200,fault_type='normal',mu=0.2,lamb=0.1,C0=0.0,annot='mu')
diffs_30 = fric_strength(dataset_1,max_depth=200,fault_type='normal',mu=0.3,lamb=0.1,C0=0.0,annot='mu')
diffs_73 = fric_strength(dataset_1,max_depth=200,fault_type='normal',mu=0.73,lamb=0.1,C0=0.0,annot='mu')
dataset_1['diffs_10_MPa'] = diffs_10
dataset_1['diffs_20_MPa'] = diffs_20
dataset_1['diffs_30_MPa'] = diffs_30
dataset_1['diffs_73_MPa'] = diffs_73
dataset_1


fault type: normal
Coefficient of friction = 0.1
Coefficient of fluid pressure = 0.1
Internal cohesion (or frictional cohesive strength) = 0.0
 

fault type: normal
Coefficient of friction = 0.2
Coefficient of fluid pressure = 0.1
Internal cohesion (or frictional cohesive strength) = 0.0
 

fault type: normal
Coefficient of friction = 0.3
Coefficient of fluid pressure = 0.1
Internal cohesion (or frictional cohesive strength) = 0.0
 

fault type: normal
Coefficient of friction = 0.73
Coefficient of fluid pressure = 0.1
Internal cohesion (or frictional cohesive strength) = 0.0
 


,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa
0,0.000000,2700.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000
1,0.010001,2700.000000,0.264793,0.043134,0.078148,0.106388,0.176788
2,0.020001,2700.000000,0.529586,0.086268,0.156296,0.212775,0.353577
3,0.030002,2700.000000,0.794378,0.129402,0.234444,0.319163,0.530365
4,0.040002,2700.000000,1.059171,0.172537,0.312593,0.425550,0.707153
...,...,...,...,...,...,...,...
19995,199.959998,3130.502626,6138.720664,999.983853,1811.716977,2466.394019,4098.503666
19996,199.969998,3130.507351,6139.036944,1000.035374,1811.810320,2466.521093,4098.714830
19997,199.979999,3130.512077,6139.353224,1000.086895,1811.903664,2466.648167,4098.925994
19998,199.989999,3130.516802,6139.669504,1000.138417,1811.997008,2466.775241,4099.137158


In [90]:
# Estimate stable geotherm for the crust: call the "turcotte_schubert_model" and pass the different arguments in order
T_crust = turcotte_schubert_model(depths[depths <= moho],thermal=(T0, heat_crust.Jq, heat_crust.A, heat_crust.K))

# Estimate average thermal crustal gradient:
Tg_crust = ((heat_crust.Jq / 1000) / heat_crust.K)*1000


#Retrieving Temp values at Moho from dataset_1 and converting to degrees C
T_at_moho_index = int(np.argwhere(depths <= moho)[-1])
T_at_moho = T_crust[T_at_moho_index]

print(' ')
print('ACCORDING TO THE MODEL:')
print('The temperature at the base of the Moho is {val} [K]' .format(val=round(T_at_moho, 1)), ('({val} [deg C])' .format(val=round(T_at_moho-273.15, 1))))

 
ACCORDING TO THE MODEL:
The temperature at the base of the Moho is 746.1 [K] (472.9 [deg C])


In [91]:
# Estimate stable geotherm for the Paleoproterozoic crust: call the "turcotte_schubert_model" and pass the different arguments in order for the lithospheric mantle
T_mantle = turcotte_schubert_model(depths[depths > moho],thermal=(T_at_moho, heat_mantle.Jq, heat_mantle.A, heat_mantle.K))

# Estimate average thermal mantle gradient (K Km**-1)
Tg_mantle = ((heat_mantle.Jq / 1000) / heat_mantle.K)*1000

# Concatenating T_crust and T_mantle values
T_values = np.hstack((T_crust, T_mantle))

# Adding temp values in Kelvin (T_values) of the whole lithopshere to dataset_1
dataset_1['T_values_K'] = T_values

print(' ')
print('ACCORDING TO THE MODEL:')
print('The temperature at the lithosphere-asthenosphere boundary is {val} [deg C]' .format(val=round(T_values[-1] - 273.15, 1)))
print('The average temperature gradient in the crust is {val} [K Km**-1]' .format(val=round(Tg_crust, 2)))
print('The average temperature gradient in the lithospheric mantle is {val} [K Km**-1]' .format(val=round(Tg_mantle, 2)))
dataset_1.round(2)

 
ACCORDING TO THE MODEL:
The temperature at the lithosphere-asthenosphere boundary is 1352.9 [deg C]
The average temperature gradient in the crust is 18.18 [K Km**-1]
The average temperature gradient in the lithospheric mantle is 7.5 [K Km**-1]


,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K
0,0.00,2700.00,0.00,-0.00,-0.00,-0.00,-0.00,280.65
1,0.01,2700.00,0.26,0.04,0.08,0.11,0.18,280.83
2,0.02,2700.00,0.53,0.09,0.16,0.21,0.35,281.01
3,0.03,2700.00,0.79,0.13,0.23,0.32,0.53,281.20
4,0.04,2700.00,1.06,0.17,0.31,0.43,0.71,281.38
...,...,...,...,...,...,...,...,...
19995,199.96,3130.50,6138.72,999.98,1811.72,2466.39,4098.50,1625.92
19996,199.97,3130.51,6139.04,1000.04,1811.81,2466.52,4098.71,1625.95
19997,199.98,3130.51,6139.35,1000.09,1811.90,2466.65,4098.93,1625.99
19998,199.99,3130.52,6139.67,1000.14,1812.00,2466.78,4099.14,1626.02


In [92]:
# Exporting dataset_1 to a .csv file. Uncomment to write output to file.
#dataset_1.to_csv('Paleoproterozoic_lithosphere_model.csv', index=True)

In [93]:
dataset_1_crust = dataset_1.iloc[0:4000, :].copy()
dataset_1_crust

,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K
0,0.000000,2700.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,280.650000
1,0.010001,2700.000000,0.264793,0.043134,0.078148,0.106388,0.176788,280.831811
2,0.020001,2700.000000,0.529586,0.086268,0.156296,0.212775,0.353577,281.013589
3,0.030002,2700.000000,0.794378,0.129402,0.234444,0.319163,0.530365,281.195335
4,0.040002,2700.000000,1.059171,0.172537,0.312593,0.425550,0.707153,281.377047
...,...,...,...,...,...,...,...,...
3995,39.951998,2752.415519,1078.383347,175.666233,318.262635,433.269142,719.980325,745.859793
3996,39.961998,2752.439940,1078.662850,175.711763,318.345125,433.381440,720.166935,745.910845
3997,39.971999,2752.464348,1078.942354,175.757294,318.427615,433.493738,720.353545,745.961865
3998,39.981999,2752.488744,1079.221857,175.802824,318.510105,433.606036,720.540155,746.012851


In [94]:
dataset_1_mantle = dataset_1.iloc[4001:20000, :].copy()
dataset_1_mantle

,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K
4001,40.012001,2752.749313,1080.133921,175.951398,318.779281,433.972482,721.149092,746.138808
4002,40.022001,2752.867316,1080.450201,176.002919,318.872625,434.099556,721.360256,746.213808
4003,40.032002,2752.985261,1080.766482,176.054440,318.965969,434.226630,721.571420,746.288806
4004,40.042002,2753.103147,1081.082762,176.105962,319.059312,434.353704,721.782584,746.363801
4005,40.052003,2753.220974,1081.399042,176.157483,319.152656,434.480778,721.993748,746.438793
...,...,...,...,...,...,...,...,...
19995,199.959998,3130.502626,6138.720664,999.983853,1811.716977,2466.394019,4098.503666,1625.916776
19996,199.969998,3130.507351,6139.036944,1000.035374,1811.810320,2466.521093,4098.714830,1625.951787
19997,199.979999,3130.512077,6139.353224,1000.086895,1811.903664,2466.648167,4098.925994,1625.986796
19998,199.989999,3130.516802,6139.669504,1000.138417,1811.997008,2466.775241,4099.137158,1626.021802


In [95]:
quartz()

Available flow laws:
'HTD' from Hirth et al. (2004)
'LP_wet' from Luan and Paterson (1992)
'GT_wet' from Gleason and Tullis (1995)
'RD_wet' from Rybacki and Dresen (2000)
'HK_wet' from Holyoke and Kronenberg (2010)
'RB_wet' from Rutter and Brodie (2004)


In [96]:
Hirth = quartz('HTD')
Luan = quartz('LP_wet')
Gleason = quartz('GT_wet')
Rybacki = quartz('RD_wet')
Holyoke = quartz('HK_wet')
Rutter = quartz('RB_wet')

# check that we obtained the three values
print(' ')
print('Checking the three values of each power law considered for the modeling:')
print(Hirth)
print(Luan)
print(Gleason)
print(Rybacki)
print(Holyoke)
print(Rutter)
print(' ')

 
Checking the three values of each power law considered for the modeling:
namespace(A=6.309573444801943e-12, E=135000, n=4.0, ref='Hirth et al. (2004)')
namespace(A=6.30957344480193e-08, E=152000, n=4.0, ref='Luan and Paterson (1992)')
namespace(A=0.00011, E=223000, n=4.0, ref='Gleason and Tullis (1995)')
namespace(A=398.1071705534973, E=356000, n=3, ref='Rybacki and Dresen (2000)')
namespace(A=0.00051, E=223000, n=4.0, ref='Holyoke and Kronenberg (2010)')
namespace(A=1.1748975549395302e-05, E=242000, n=2.97, ref='Rutter and Brodie (2004)')
 


Estimate the differential stress at steady-state using 1.0e-15 $s^{-1}$ as the reference average shear strain rate in the ductile lithosphere.

In [97]:
dataset_1_crust['model_HTD'] = power_law_creep(strain_rate=1e-15,
                                       n=Hirth.n,
                                       A=Hirth.A,
                                       E=Hirth.E,
                                       T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust['model_LP_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Luan.n,
                                      A=Luan.A,
                                      E=Luan.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust['model_GT_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Gleason.n,
                                      A=Gleason.A,
                                      E=Gleason.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust['model_RD_wet'] = power_law_creep(strain_rate = 1e-15,
                                      n=Rybacki.n,
                                      A=Rybacki.A,
                                      E=Rybacki.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )                                                 

dataset_1_crust['model_HK_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Holyoke.n,
                                      A=Holyoke.A,
                                      E=Holyoke.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust['model_RB_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Rutter.n,
                                      A=Rutter.A,
                                      E=Rutter.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust.round(2)

,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K,model_HTD,model_LP_wet,model_GT_wet,model_RD_wet,model_HK_wet,model_RB_wet
0,0.00,2700.00,0.00,-0.00,-0.00,-0.00,-0.00,280.65,214503.72,132565.44,41271625.34,1.656412e+16,28125949.96,5.949533e+11
1,0.01,2700.00,0.26,0.04,0.08,0.11,0.18,280.83,212504.54,131175.16,40638169.34,1.602765e+16,27694259.86,5.816544e+11
2,0.02,2700.00,0.53,0.09,0.16,0.21,0.35,281.01,210526.90,129801.49,40015347.98,1.550932e+16,27269817.11,5.686717e+11
3,0.03,2700.00,0.79,0.13,0.23,0.32,0.53,281.20,208570.54,128444.18,39402968.17,1.500847e+16,26852490.10,5.559972e+11
4,0.04,2700.00,1.06,0.17,0.31,0.43,0.71,281.38,206635.19,127103.04,38800840.54,1.452449e+16,26442149.79,5.436233e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,39.95,2752.42,1078.38,175.67,318.26,433.27,719.98,745.86,25.92,5.14,13.93,2.778000e+02,9.49,2.068700e+02
3996,39.96,2752.44,1078.66,175.71,318.35,433.38,720.17,745.91,25.91,5.14,13.92,2.774300e+02,9.49,2.066800e+02
3997,39.97,2752.46,1078.94,175.76,318.43,433.49,720.35,745.96,25.90,5.14,13.91,2.770700e+02,9.48,2.065000e+02
3998,39.98,2752.49,1079.22,175.80,318.51,433.61,720.54,746.01,25.89,5.14,13.90,2.767100e+02,9.47,2.063100e+02


In [98]:
# Exporting dataset_1_crust to a .csv file. Uncomment to write output to file.
#dataset_1_crust.to_csv('Paleoproterozoic_crust_model.csv', index=True)

In [99]:
olivine()

Available flow laws:
'KW_wet' from Karato and Wu (1993)
'HK_wet' from Hirth and Kohlstedt (2003)
'HK_dry' from Hirth and Kohlstedt (2003)
'KJ_wet' from Karato and Jung (2003)
'KJ_dry' from Karato and Jung (2003)
'ZK_dry' (dry peridotite) from Zimmerman and Kohlstedt (2004)
'Faul_dry' from Faul et al. (2011)
'Ohuchi' from Ohuchi et al. (2015)


In [100]:
Karato_0 = olivine('KW_wet')
Hirth_2 = olivine('HK_wet')
Hirth_3 = olivine('HK_dry')
Karato_1 = olivine('KJ_wet')
Karato_2 = olivine('KJ_dry')
Zimmerman = olivine('ZK_dry')
Faul = olivine('Faul_dry')
Ohuchi = olivine('Ohuchi')

# check that we obtained the three values
print(' ')
print('Checking the three values of each power law considered for the modeling:')
print(Karato_0)
print(Hirth_2) 
print(Hirth_3)
print(Karato_1)
print(Karato_2)
print(Zimmerman)
print(Faul)
print(Ohuchi)
print(' ')

 
Checking the three values of each power law considered for the modeling:
namespace(A=1584.893192461114, E=430000, V=1.5e-05, n=3, r=nan, ref='Karato and Wu (1993)')
namespace(A=1584.893192461114, E=520000, V=2.2e-05, n=3.5, r=nan, ref='Hirth and Kohlstedt (2003)')
namespace(A=100000.0, E=530000, V=1.8e-05, n=3.5, r=nan, ref='Hirth and Kohlstedt (2003)')
namespace(A=794.3282347242813, E=470000, V=2.4e-05, n=3.0, r=nan, ref='Karato and Jung (2003)')
namespace(A=1258925.411794166, E=510000, V=1.4e-05, n=3.0, r=nan, ref='Karato and Jung (2003)')
namespace(A=63095.7344480193, E=550000, V=nan, n=4.3, r=nan, ref='Zimmerman and Kohlstedt (2004)')
namespace(A=0.3, E=682000, V=nan, n=8.2, r=nan, ref='Faul et al. (2011)')
namespace(A=1.2882495516931348e-05, E=423000, V=1.76e-05, n=3.0, r=1.25, ref='Ohuchi et al. (2015)')
 


In [101]:
dataset_1_mantle['model_KW_wet'] = power_law_creep(strain_rate=1e-15,
                                                  n=Karato_0.n,
                                                  A=Karato_0.A,
                                                  E=Karato_0.E,
                                                  T=dataset_1_mantle['T_values_K'])

dataset_1_mantle['model_HK_wet'] = power_law_creep(strain_rate=1e-15,
                                       n=Hirth_2.n,
                                       A=Hirth_2.A,
                                       E=Hirth_2.E,
                                       T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_HK_dry'] = power_law_creep(strain_rate=1e-15,
                                      n=Hirth_3.n,
                                      A=Hirth_3.A,
                                      E=Hirth_3.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_KJ_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Karato_1.n,
                                      A=Karato_1.A,
                                      E=Karato_1.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_KJ_dry'] = power_law_creep(strain_rate=1e-15,
                                      n=Karato_2.n,
                                      A=Karato_2.A,
                                      E=Karato_2.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_ZK_dry'] = power_law_creep(strain_rate=1e-15,
                                      n=Zimmerman.n,
                                      A=Zimmerman.A,
                                      E=Zimmerman.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_Faul_dry'] = power_law_creep(strain_rate=1e-15,
                                      n=Faul.n,
                                      A=Faul.A,
                                      E=Faul.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_Ohuchi'] = power_law_creep(strain_rate=1e-15,
                                      n=Ohuchi.n,
                                      A=Ohuchi.A,
                                      E=Ohuchi.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle.round(2)

,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K,model_KW_wet,model_HK_wet,model_HK_dry,model_KJ_wet,model_KJ_dry,model_ZK_dry,model_Faul_dry,model_Ohuchi
4001,40.01,2752.75,1080.13,175.95,318.78,433.97,721.15,746.14,9277.19,158773.24,77002.80,100188.91,73715.03,22361.78,11399.45,3167653.62
4002,40.02,2752.87,1080.45,176.00,318.87,434.10,721.36,746.21,9255.67,158391.53,76814.12,99934.94,73512.29,22315.49,11384.10,3160425.84
4003,40.03,2752.99,1080.77,176.05,318.97,434.23,721.57,746.29,9234.21,158010.82,76625.95,99681.66,73310.15,22269.31,11368.77,3153216.24
4004,40.04,2753.10,1081.08,176.11,319.06,434.35,721.78,746.36,9212.80,157631.12,76438.28,99429.09,73108.61,22223.23,11353.47,3146024.77
4005,40.05,2753.22,1081.40,176.16,319.15,434.48,721.99,746.44,9191.45,157252.42,76251.11,99177.22,72907.67,22177.26,11338.19,3138851.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,199.96,3130.50,6138.72,999.98,1811.72,2466.39,4098.50,1625.92,0.03,0.37,0.14,0.12,0.03,0.32,8.06,14.44
19996,199.97,3130.51,6139.04,1000.04,1811.81,2466.52,4098.71,1625.95,0.03,0.37,0.14,0.12,0.03,0.32,8.06,14.44
19997,199.98,3130.51,6139.35,1000.09,1811.90,2466.65,4098.93,1625.99,0.03,0.37,0.14,0.12,0.03,0.32,8.06,14.44
19998,199.99,3130.52,6139.67,1000.14,1812.00,2466.78,4099.14,1626.02,0.03,0.37,0.14,0.12,0.03,0.32,8.06,14.44


In [102]:
# Exporting dataset_1_mantle to a .csv file. Uncomment to write output to file.
#dataset_1_mantle.to_csv('Paleoproterozoic_mantle_model.csv', index=True)

### Strength envelope calculated for the Paleoproterozoic lithosphere type:

In [103]:
fig = go.Figure()

# plot the limits of the lithosheric layers (i.e., Moho and LAB depths)
# crust area
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [moho, moho],
    mode='lines',
    line=dict(color='#F5F5DC'),
    fill = 'tozeroy',
    showlegend = False
))
# lithospheric mantle area
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [Lab, Lab],
    mode='lines', 
    line=dict(color='#70A459'),
    fill='tonexty',
    showlegend = False
))
# asthernosphere area
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [230, 230],
    mode= 'lines',
    line=dict(color='#fdbcb4'),
    fill='tonexty',
    showlegend = False
))

#moho line
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [Lab, Lab],
    mode='lines', 
    line=dict(color='white', width=1),
    showlegend = False
))
# lithosphere line
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [moho, moho],
    mode='lines',
    line=dict(color='white', width=1),
    showlegend = False
))
#TEXT
# moho text
fig.add_trace(go.Scatter(
    x = [100],
    y = [moho - moho/90],
    mode='text',
    text = 'Moho',
    name = 'moho text',
    showlegend = False
))
# lab text
fig.add_trace(go.Scatter(
    x = [310],
    y = [Lab - Lab/90],
    mode='text',
    text = 'Lithosphere base',
    name = 'litho text',
    showlegend = False
))
#asthenosphere text
fig.add_trace(go.Scatter(
    x = [300],
    y = [215 - 215/90],
    mode= 'text',
    text = 'Asthenosphere',
    name = 'Astheno text',
    showlegend = False
))
# plot frictional strength slopes in the depth vs differential stress space
fig.add_trace(go.Scatter(
    x = dataset_1['diffs_10_MPa']/1000,
    y = dataset_1['depths_km'],
    name = 'angle of internal friction = 10',
    mode='lines',
    xaxis='x2'
))
fig.add_trace(go.Scatter(
    x = dataset_1['diffs_20_MPa']/1000,
    y = dataset_1['depths_km'],
    name = 'angle of internal friction = 20',
    mode='lines',
    xaxis='x2'
))
fig.add_trace(go.Scatter(
    x = dataset_1['diffs_30_MPa']/1000,
    y = dataset_1['depths_km'],
    name = 'angle of internal friction = 30',
    mode='lines',
    xaxis='x2'
))

# plot the strength envelopes for the crust
fig.add_trace(go.Scatter(
    x = dataset_1_crust['model_GT_wet']/1000,
    y = dataset_1_crust['depths_km'],
    name = "Gleason and Tullis (1995)",
    mode = 'lines',
    xaxis = 'x2'
))
fig.add_trace(go.Scatter(
    x = dataset_1_crust['model_RD_wet']/1000,
    y = dataset_1_crust['depths_km'],
    name = "Rybacki and Dresen (2000)",
    mode = 'lines',
    xaxis = 'x2'
))

#plot geothermal gradient of the Paleoproterozoic lithosphere
fig.add_trace(go.Scatter(
    x = dataset_1['T_values_K']-273,
    y = dataset_1['depths_km'],
    name = "Geothermal gradient",
    line=dict(color='red', width=3.5),
    mode = 'lines',
    xaxis = 'x'
))

# plot the strength envelopes for the mantle
author = 'model_Ohuchi'
legend = 'Ohuchi et al., (2015)'
fig.add_trace(go.Scatter(
    x = dataset_1_mantle['{}'.format(author)]/1000,
    y = dataset_1_mantle['depths_km'],
    name = legend, 
    mode='lines',
    xaxis = 'x2'
))


fig.update_layout(
    autosize = False,
    width = 800, #800
    height = 1200, #1200
    yaxis = dict(
        autorange = 'reversed',
        range = [0, 1100]
    ),
    xaxis =dict(
        title = 'Temperature [ºC]',
        range = [0, 2500]
    ),
    xaxis2 = dict(
        title = "Differential Stress [GPa]",
        overlaying = 'x',
        side = 'top',
        range = [0, 2]
    )
)
#fig.show(renderer="colab") #Uncomment to show graphical representation in Google Colab platform

In [104]:
# Exporting Paleoproterozoic lithospheric strength envelope to pdf file. Uncomment to write output to file. 
#fig.write_image("Paleoproterozoic_lithospheric_strength_envelope.pdf")

## Archean lithospheric model set up:

In [105]:
moho = 31.1      # Archean crust thickness [km], average from Christensen and Mooney (1995) and Svartman Dias et al. (2015)
Lab = 250        # Archean lithosphere thickness [km] from Artemieva and Mooney (2001)
T0 = 7.5 + 273.15         # surface temperature in K (taken from the KTB superdeep borehole)
crust_densities = (2700, 2828, 2850)      # Kg cm**-3
crust_layers = (0.2, 0.9)      # depth of the base of the crustal layer with respect to the depth of the moho
mantle_density = 3200      # Kg cm**-3

#Set the average heat parameters for the crust.
#the thermal parameters are stored within two different objects called heat_crust and heat_mantle:
#Jq : average heat flux [mW m**-2]
#A : average heat productivity [microW m**-3]
#K : coefficient of thermal conductivity [W m**-1 K**-1]

heat_crust = SimpleNamespace(Jq=30, A=0.45, K=2.7)      # crustal and mantle thermal parameters for Archean terranes. Average heat flux (Jq) obtained from Artemieva (2011), table 4.29. Average heat productivity (A) value obtained from Artemieva (2011), table 4.8. Coefficient of thermal conductivity (K) obtained from Artemiva (2011), table 4.3.
heat_mantle = SimpleNamespace(Jq=20, A=0.03, K=4)

g = 9.80665      # average gravitational acceleration [m s**-2]

print(' ')
print('THE MODELLED ARCHEAN LITHOSPHERE COMPRISES:')
print('a {val} [km] thick Upper Crust' .format(val=round(crust_layers[0]*moho)))
print('a {val} [km] thick Middle Crust' .format(val=round(crust_layers[1]*moho-crust_layers[0]*moho)))
print('a {val} [km] thick Lower Crust' .format(val=round(moho-crust_layers[1]*moho)))
print('and a {val} [km] thick lithospheric mantle' .format(val=round(Lab-moho)))

 
THE MODELLED ARCHEAN LITHOSPHERE COMPRISES:
a 6 [km] thick Upper Crust
a 22 [km] thick Middle Crust
a 3 [km] thick Lower Crust
and a 219 [km] thick lithospheric mantle


In [106]:
 # Estimate a mesh density so that there is a measure every 10 m
mesh_density = Lab*100

 # Get a list of depths [km] and corresponding average density [kg m**-3] and pressures (MPa)
depths = np.linspace(0, Lab, mesh_density)
densities = calc_average_density(depths, moho, Lab, crust_densities, crust_layers, mantle_density)
pressures = densities * g * depths / 1e3      # /1e3 to obtain MPa
dataset_1 = pd.DataFrame({'depths_km': depths,
                          'mean_density': densities,
                         'Pressure_MPa': pressures})      #dataframe containing the different parameters of the lithospheric model 
dataset_1.round(2)

,depths_km,mean_density,Pressure_MPa
0,0.00,2700.00,0.00
1,0.01,2700.00,0.26
2,0.02,2700.00,0.53
3,0.03,2700.00,0.79
4,0.04,2700.00,1.06
...,...,...,...
24995,249.96,3150.82,7723.52
24996,249.97,3150.82,7723.83
24997,249.98,3150.83,7724.14
24998,249.99,3150.83,7724.46


Calculating the frictional strength slopes in the depth versus differential stress space for different coefficients of friction (mu):

In [107]:
diffs_10 = fric_strength(dataset_1,max_depth=250,fault_type='normal',mu=0.1,lamb=0.1,C0=0.0,annot='mu')
diffs_20 = fric_strength(dataset_1,max_depth=250,fault_type='normal',mu=0.2,lamb=0.1,C0=0.0,annot='mu')
diffs_30 = fric_strength(dataset_1,max_depth=250,fault_type='normal',mu=0.3,lamb=0.1,C0=0.0,annot='mu')
diffs_73 = fric_strength(dataset_1,max_depth=250,fault_type='normal',mu=0.73,lamb=0.1,C0=0.0,annot='mu')
dataset_1['diffs_10_MPa'] = diffs_10
dataset_1['diffs_20_MPa'] = diffs_20
dataset_1['diffs_30_MPa'] = diffs_30
dataset_1['diffs_73_MPa'] = diffs_73
dataset_1


fault type: normal
Coefficient of friction = 0.1
Coefficient of fluid pressure = 0.1
Internal cohesion (or frictional cohesive strength) = 0.0
 

fault type: normal
Coefficient of friction = 0.2
Coefficient of fluid pressure = 0.1
Internal cohesion (or frictional cohesive strength) = 0.0
 

fault type: normal
Coefficient of friction = 0.3
Coefficient of fluid pressure = 0.1
Internal cohesion (or frictional cohesive strength) = 0.0
 

fault type: normal
Coefficient of friction = 0.73
Coefficient of fluid pressure = 0.1
Internal cohesion (or frictional cohesive strength) = 0.0
 


,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa
0,0.000000,2700.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000
1,0.010000,2700.000000,0.264790,0.043134,0.078147,0.106386,0.176787
2,0.020001,2700.000000,0.529580,0.086267,0.156295,0.212773,0.353573
3,0.030001,2700.000000,0.794370,0.129401,0.234442,0.319159,0.530360
4,0.040002,2700.000000,1.059161,0.172535,0.312589,0.425546,0.707146
...,...,...,...,...,...,...,...
24995,249.959998,3150.822404,7723.517123,1258.143648,2279.437013,3103.128075,5156.589619
24996,249.969999,3150.824372,7723.830949,1258.194769,2279.529632,3103.254163,5156.799143
24997,249.979999,3150.826339,7724.144774,1258.245891,2279.622251,3103.380250,5157.008668
24998,249.990000,3150.828306,7724.458599,1258.297012,2279.714870,3103.506338,5157.218193


In [108]:
# Estimate stable geotherm for the Archean crust: call the "turcotte_schubert_model" and pass the different arguments in order
T_crust = turcotte_schubert_model(depths[depths <= moho],thermal=(T0, heat_crust.Jq, heat_crust.A, heat_crust.K))

# Estimate average thermal gradient of the crust (K Km**-1)
Tg_crust = ((heat_crust.Jq/1000) / heat_crust.K)*1000

#Retrieving Temp values at Moho from dataset_1 and converting to degrees C
T_at_moho_index = int(np.argwhere(depths <= moho)[-1])
T_at_moho = T_crust[T_at_moho_index]

print(' ')
print('ACCORDING TO THE MODEL:')
print('The temperature at the base of the Moho is {val} [K]' .format(val=round(T_at_moho, 1)), ('({val} [deg C])' .format(val=round(T_at_moho-273.15, 1))))

 
ACCORDING TO THE MODEL:
The temperature at the base of the Moho is 545.6 [K] (272.4 [deg C])


In [109]:
# Estimate stable geotherm: call the "turcotte_schubert_model" and pass the different arguments in order for the lithospheric mantle
T_mantle = turcotte_schubert_model(depths[depths > moho],thermal=(T_at_moho, heat_mantle.Jq, heat_mantle.A, heat_mantle.K))

# Estimate average thermal gradient of the mantle (K Km**-1)
Tg_mantle = ((heat_mantle.Jq/1000) / heat_mantle.K)*1000

# Concatenating T_crust and T_mantle values
T_values = np.hstack((T_crust, T_mantle))

# Adding temp values in Kelvin (T_values) of the whole lithopshere to dataset_1
dataset_1['T_values_K'] = T_values

print(' ')
print('ACCORDING TO THE MODEL:')
print('The temperature at the lithosphere-asthenosphere boundary is {val} [deg C]' .format(val=round(T_values[-1] - 273.15, 1)))
print('The average temperature gradient in the crust is {val} [K Km**-1]' .format(val=round(Tg_crust, 2)))
print('The average temperature gradient in the lithospheric mantle is {val} [K Km**-1]' .format(val=round(Tg_mantle, 2)))
dataset_1.round(2)

 
ACCORDING TO THE MODEL:
The temperature at the lithosphere-asthenosphere boundary is 1187.2 [deg C]
The average temperature gradient in the crust is 11.11 [K Km**-1]
The average temperature gradient in the lithospheric mantle is 5.0 [K Km**-1]


,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K
0,0.00,2700.00,0.00,-0.00,-0.00,-0.00,-0.00,280.65
1,0.01,2700.00,0.26,0.04,0.08,0.11,0.18,280.76
2,0.02,2700.00,0.53,0.09,0.16,0.21,0.35,280.87
3,0.03,2700.00,0.79,0.13,0.23,0.32,0.53,280.98
4,0.04,2700.00,1.06,0.17,0.31,0.43,0.71,281.09
...,...,...,...,...,...,...,...,...
24995,249.96,3150.82,7723.52,1258.14,2279.44,3103.13,5156.59,1460.22
24996,249.97,3150.82,7723.83,1258.19,2279.53,3103.25,5156.80,1460.26
24997,249.98,3150.83,7724.14,1258.25,2279.62,3103.38,5157.01,1460.29
24998,249.99,3150.83,7724.46,1258.30,2279.71,3103.51,5157.22,1460.33


In [110]:
# Exporting dataset_1 to a .csv file. Uncomment to write output to file.
#dataset_1.to_csv('Archean_lithosphere_model.csv', index=True)

In [111]:
dataset_1_crust = dataset_1.iloc[0:3110, :].copy()
dataset_1_crust

,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K
0,0.000000,2700.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,280.650000
1,0.010000,2700.000000,0.264790,0.043134,0.078147,0.106386,0.176787,280.761107
2,0.020001,2700.000000,0.529580,0.086267,0.156295,0.212773,0.353573,280.872198
3,0.030001,2700.000000,0.794370,0.129401,0.234442,0.319159,0.530360,280.983272
4,0.040002,2700.000000,1.059161,0.172535,0.312589,0.425546,0.707146,281.094329
...,...,...,...,...,...,...,...,...
3105,31.051242,2804.575201,854.017444,139.117530,252.045660,343.124184,570.182912,545.315498
3106,31.061242,2804.589826,854.296945,139.163060,252.128149,343.236481,570.369521,545.374851
3107,31.071243,2804.604442,854.576446,139.208590,252.210638,343.348777,570.556129,545.434187
3108,31.081243,2804.619048,854.855947,139.254120,252.293127,343.461074,570.742737,545.493507


In [112]:
dataset_1_mantle = dataset_1.iloc[3111:25000, :].copy()
dataset_1_mantle

,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K
3111,31.111244,2804.887817,855.763098,139.401893,252.560854,343.825546,571.348394,545.602812
3112,31.121245,2805.014781,856.076923,139.453014,252.653473,343.951634,571.557919,545.652813
3113,31.131245,2805.141664,856.390749,139.504136,252.746092,344.077722,571.767444,545.702813
3114,31.141246,2805.268465,856.704574,139.555257,252.838711,344.203809,571.976969,545.752812
3115,31.151246,2805.395185,857.018399,139.606379,252.931330,344.329897,572.186494,545.802811
...,...,...,...,...,...,...,...,...
24995,249.959998,3150.822404,7723.517123,1258.143648,2279.437013,3103.128075,5156.589619,1460.224753
24996,249.969999,3150.824372,7723.830949,1258.194769,2279.529632,3103.254163,5156.799143,1460.258340
24997,249.979999,3150.826339,7724.144774,1258.245891,2279.622251,3103.380250,5157.008668,1460.291925
24998,249.990000,3150.828306,7724.458599,1258.297012,2279.714870,3103.506338,5157.218193,1460.325510


In [113]:
quartz()

Available flow laws:
'HTD' from Hirth et al. (2004)
'LP_wet' from Luan and Paterson (1992)
'GT_wet' from Gleason and Tullis (1995)
'RD_wet' from Rybacki and Dresen (2000)
'HK_wet' from Holyoke and Kronenberg (2010)
'RB_wet' from Rutter and Brodie (2004)


In [114]:
Hirth = quartz('HTD')
Luan = quartz('LP_wet')
Gleason = quartz('GT_wet')
Rybacki = quartz('RD_wet')
Holyoke = quartz('HK_wet')
Rutter = quartz('RB_wet')

# check that we obtained the three values
print(' ')
print('Checking the three values of each power law considered for the modeling:')
print(Hirth)
print(Luan)
print(Gleason)
print(Rybacki)
print(Holyoke)
print(Rutter)
print(' ')

 
Checking the three values of each power law considered for the modeling:
namespace(A=6.309573444801943e-12, E=135000, n=4.0, ref='Hirth et al. (2004)')
namespace(A=6.30957344480193e-08, E=152000, n=4.0, ref='Luan and Paterson (1992)')
namespace(A=0.00011, E=223000, n=4.0, ref='Gleason and Tullis (1995)')
namespace(A=398.1071705534973, E=356000, n=3, ref='Rybacki and Dresen (2000)')
namespace(A=0.00051, E=223000, n=4.0, ref='Holyoke and Kronenberg (2010)')
namespace(A=1.1748975549395302e-05, E=242000, n=2.97, ref='Rutter and Brodie (2004)')
 


Estimate the differential stress at steady-state using 1.0e-15 $s^{-1}$ as the reference average shear strain rate in the ductile lithosphere.

In [115]:
dataset_1_crust['model_HTD'] = power_law_creep(strain_rate=1e-15,
                                       n=Hirth.n,
                                       A=Hirth.A,
                                       E=Hirth.E,
                                       T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust['model_LP_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Luan.n,
                                      A=Luan.A,
                                      E=Luan.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust['model_GT_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Gleason.n,
                                      A=Gleason.A,
                                      E=Gleason.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust['model_RD_wet'] = power_law_creep(strain_rate = 1e-15,
                                      n=Rybacki.n,
                                      A=Rybacki.A,
                                      E=Rybacki.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )                                                 

dataset_1_crust['model_HK_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Holyoke.n,
                                      A=Holyoke.A,
                                      E=Holyoke.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust['model_RB_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Rutter.n,
                                      A=Rutter.A,
                                      E=Rutter.E,
                                      T=dataset_1_crust['T_values_K'],
                                      )

dataset_1_crust.round(2)

,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K,model_HTD,model_LP_wet,model_GT_wet,model_RD_wet,model_HK_wet,model_RB_wet
0,0.00,2700.00,0.00,-0.00,-0.00,-0.00,-0.00,280.65,214503.72,132565.44,41271625.34,1.656412e+16,28125949.96,5.949533e+11
1,0.01,2700.00,0.26,0.04,0.08,0.11,0.18,280.76,213279.46,131713.87,40883250.50,1.623410e+16,27861278.73,5.867884e+11
2,0.02,2700.00,0.53,0.09,0.16,0.21,0.35,280.87,212063.33,130868.56,40498890.65,1.591096e+16,27599343.67,5.787431e+11
3,0.03,2700.00,0.79,0.13,0.23,0.32,0.53,280.98,210855.28,130029.46,40118500.77,1.559454e+16,27340114.08,5.708155e+11
4,0.04,2700.00,1.06,0.17,0.31,0.43,0.71,281.09,209655.22,129196.53,39742036.35,1.528470e+16,27083559.62,5.630038e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3105,31.05,2804.58,854.02,139.12,252.05,343.12,570.18,545.32,191.77,48.96,379.95,3.161879e+05,258.93,2.595478e+04
3106,31.06,2804.59,854.30,139.16,252.13,343.24,570.37,545.37,191.61,48.92,379.44,3.152885e+05,258.58,2.590406e+04
3107,31.07,2804.60,854.58,139.21,252.21,343.35,570.56,545.43,191.46,48.87,378.93,3.143922e+05,258.24,2.585347e+04
3108,31.08,2804.62,854.86,139.25,252.29,343.46,570.74,545.49,191.30,48.83,378.43,3.134988e+05,257.89,2.580301e+04


In [116]:
# Exporting dataset_1_crust to a .csv file. Uncomment to write output to file.
#dataset_1_crust.to_csv('Archean_crust_model.csv', index=True)

In [117]:
olivine()

Available flow laws:
'KW_wet' from Karato and Wu (1993)
'HK_wet' from Hirth and Kohlstedt (2003)
'HK_dry' from Hirth and Kohlstedt (2003)
'KJ_wet' from Karato and Jung (2003)
'KJ_dry' from Karato and Jung (2003)
'ZK_dry' (dry peridotite) from Zimmerman and Kohlstedt (2004)
'Faul_dry' from Faul et al. (2011)
'Ohuchi' from Ohuchi et al. (2015)


In [118]:
Karato_0 = olivine('KW_wet')
Hirth_2 = olivine('HK_wet')
Hirth_3 = olivine('HK_dry')
Karato_1 = olivine('KJ_wet')
Karato_2 = olivine('KJ_dry')
Zimmerman = olivine('ZK_dry')
Faul = olivine('Faul_dry')
Ohuchi = olivine('Ohuchi')

# check that we obtained the three values
print(' ')
print('Checking the three values of each power law considered for the modeling:')
print(Karato_0)
print(Hirth_2) 
print(Hirth_3)
print(Karato_1)
print(Karato_2)
print(Zimmerman)
print(Faul)
print(Ohuchi)
print(' ')

 
Checking the three values of each power law considered for the modeling:
namespace(A=1584.893192461114, E=430000, V=1.5e-05, n=3, r=nan, ref='Karato and Wu (1993)')
namespace(A=1584.893192461114, E=520000, V=2.2e-05, n=3.5, r=nan, ref='Hirth and Kohlstedt (2003)')
namespace(A=100000.0, E=530000, V=1.8e-05, n=3.5, r=nan, ref='Hirth and Kohlstedt (2003)')
namespace(A=794.3282347242813, E=470000, V=2.4e-05, n=3.0, r=nan, ref='Karato and Jung (2003)')
namespace(A=1258925.411794166, E=510000, V=1.4e-05, n=3.0, r=nan, ref='Karato and Jung (2003)')
namespace(A=63095.7344480193, E=550000, V=nan, n=4.3, r=nan, ref='Zimmerman and Kohlstedt (2004)')
namespace(A=0.3, E=682000, V=nan, n=8.2, r=nan, ref='Faul et al. (2011)')
namespace(A=1.2882495516931348e-05, E=423000, V=1.76e-05, n=3.0, r=1.25, ref='Ohuchi et al. (2015)')
 


In [119]:
dataset_1_mantle['model_KW_wet'] = power_law_creep(strain_rate=1e-15,
                                                  n=Karato_0.n,
                                                  A=Karato_0.A,
                                                  E=Karato_0.E,
                                                  T=dataset_1_mantle['T_values_K'])

dataset_1_mantle['model_HK_wet'] = power_law_creep(strain_rate=1e-15,
                                       n=Hirth_2.n,
                                       A=Hirth_2.A,
                                       E=Hirth_2.E,
                                       T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_HK_dry'] = power_law_creep(strain_rate=1e-15,
                                      n=Hirth_3.n,
                                      A=Hirth_3.A,
                                      E=Hirth_3.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_KJ_wet'] = power_law_creep(strain_rate=1e-15,
                                      n=Karato_1.n,
                                      A=Karato_1.A,
                                      E=Karato_1.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_KJ_dry'] = power_law_creep(strain_rate=1e-15,
                                      n=Karato_2.n,
                                      A=Karato_2.A,
                                      E=Karato_2.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_ZK_dry'] = power_law_creep(strain_rate=1e-15,
                                      n=Zimmerman.n,
                                      A=Zimmerman.A,
                                      E=Zimmerman.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_Faul_dry'] = power_law_creep(strain_rate=1e-15,
                                      n=Faul.n,
                                      A=Faul.A,
                                      E=Faul.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle['model_Ohuchi'] = power_law_creep(strain_rate=1e-15,
                                      n=Ohuchi.n,
                                      A=Ohuchi.A,
                                      E=Ohuchi.E,
                                      T=dataset_1_mantle['T_values_K'],
                                      )

dataset_1_mantle.round(2)

,depths_km,mean_density,Pressure_MPa,diffs_10_MPa,diffs_20_MPa,diffs_30_MPa,diffs_73_MPa,T_values_K,model_KW_wet,model_HK_wet,model_HK_dry,model_KJ_wet,model_KJ_dry,model_ZK_dry,model_Faul_dry,model_Ohuchi
3111,31.11,2804.89,855.76,139.40,252.56,343.83,571.35,545.60,45231190.64,1.055789e+09,6.064871e+08,1.076249e+09,1.744701e+09,43712113.22,1573614.98,1.344998e+10
3112,31.12,2805.01,856.08,139.45,252.65,343.95,571.56,545.65,45100420.95,1.052625e+09,6.046348e+08,1.072849e+09,1.738720e+09,43599319.34,1570973.46,1.341172e+10
3113,31.13,2805.14,856.39,139.50,252.75,344.08,571.77,545.70,44970055.15,1.049471e+09,6.027885e+08,1.069460e+09,1.732761e+09,43486838.79,1568336.89,1.337359e+10
3114,31.14,2805.27,856.70,139.56,252.84,344.20,571.98,545.75,44840091.91,1.046327e+09,6.009482e+08,1.066082e+09,1.726823e+09,43374670.63,1565705.27,1.333557e+10
3115,31.15,2805.40,857.02,139.61,252.93,344.33,572.19,545.80,44710529.91,1.043194e+09,5.991139e+08,1.062715e+09,1.720907e+09,43262813.95,1563078.58,1.329766e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,249.96,3150.82,7723.52,1258.14,2279.44,3103.13,5156.59,1460.22,0.11,1.300000e+00,5.000000e-01,4.300000e-01,1.100000e-01,0.93,16.20,4.717000e+01
24996,249.97,3150.82,7723.83,1258.19,2279.53,3103.25,5156.80,1460.26,0.11,1.300000e+00,5.000000e-01,4.300000e-01,1.100000e-01,0.93,16.20,4.716000e+01
24997,249.98,3150.83,7724.14,1258.25,2279.62,3103.38,5157.01,1460.29,0.11,1.300000e+00,5.000000e-01,4.300000e-01,1.100000e-01,0.93,16.20,4.715000e+01
24998,249.99,3150.83,7724.46,1258.30,2279.71,3103.51,5157.22,1460.33,0.11,1.300000e+00,5.000000e-01,4.300000e-01,1.100000e-01,0.93,16.20,4.714000e+01


In [120]:
# Exporting dataset_1_mantle to a .csv file. Uncomment to write output to file.
#dataset_1_mantle.to_csv('Archean_mantle_model.csv', index=True)

### Strength envelope calculated for the Archean lithosphere type:

In [121]:
fig = go.Figure()

# plot the limits of the lithosheric layers (i.e., Moho and LAB depths)
# crust area
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [moho, moho],
    mode='lines',
    line=dict(color='#F5F5DC'),
    fill = 'tozeroy',
    showlegend = False
))
# lithospheric mantle area
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [Lab, Lab],
    mode='lines', 
    line=dict(color='#70A459'),
    fill='tonexty',
    showlegend = False
))
# asthernosphere area
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [280, 280],
    mode= 'lines',
    line=dict(color='#fdbcb4'),
    fill='tonexty',
    showlegend = False
))

#moho line
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [Lab, Lab],
    mode='lines', 
    line=dict(color='white', width=1),
    showlegend = False
))
# lithosphere line
fig.add_trace(go.Scatter(
    x = [0, 2500],
    y = [moho, moho],
    mode='lines',
    line=dict(color='white', width=1),
    showlegend = False
))
#TEXT
# moho text
fig.add_trace(go.Scatter(
    x = [100],
    y = [moho - moho/90],
    mode='text',
    text = 'Moho',
    name = 'moho text',
    showlegend = False
))
# lab text
fig.add_trace(go.Scatter(
    x = [300],
    y = [Lab - Lab/90],
    mode='text',
    text = 'Lithosphere base',
    name = 'litho text',
    showlegend = False
))
#asthenosphere text
fig.add_trace(go.Scatter(
    x = [300],
    y = [265 - 265/90],
    mode= 'text',
    text = 'Asthenosphere',
    name = 'Astheno text',
    showlegend = False
))
# plot frictional strength slopes in the depth vs differential stress space
fig.add_trace(go.Scatter(
    x = dataset_1['diffs_10_MPa']/1000,
    y = dataset_1['depths_km'],
    name = 'angle of internal friction = 10',
    mode='lines',
    xaxis='x2'
))
fig.add_trace(go.Scatter(
    x = dataset_1['diffs_20_MPa']/1000,
    y = dataset_1['depths_km'],
    name = 'angle of internal friction = 20',
    mode='lines',
    xaxis='x2'
))
fig.add_trace(go.Scatter(
    x = dataset_1['diffs_30_MPa']/1000,
    y = dataset_1['depths_km'],
    name = 'angle of internal friction = 30',
    mode='lines',
    xaxis='x2'
))

# plot the strength envelopes for the crust
#fig.add_trace(go.Scatter(
    #x = dataset_1['Pressure_MPa']/1000,
    #y = dataset_1['depths_km'],
    #name = "Goetze criterion",
    #mode = 'lines',
    #xaxis = 'x2'
#))

#plot geothermal gradient of the Archean lithosphere
fig.add_trace(go.Scatter(
    x = dataset_1['T_values_K']-273,
    y = dataset_1['depths_km'],
    name = "Geothermal gradient",
    line=dict(color='red', width=3.5),
    mode = 'lines',
    xaxis = 'x'
))

# plot the strength envelopes for the mantle
author = 'model_Ohuchi'
legend = 'Ohuchi et al., (2015)'
fig.add_trace(go.Scatter(
    x = dataset_1_mantle['{}'.format(author)]/1000,
    y = dataset_1_mantle['depths_km'],
    name = legend, 
    mode='lines',
    xaxis = 'x2'
))


fig.update_layout(
    autosize = False,
    width = 800, #800
    height = 1200, #1200
    yaxis = dict(
        autorange = 'reversed',
        range = [0, 1100]
    ),
    xaxis =dict(
        title = 'Temperature [ºC]',
        range = [0, 2500]
    ),
    xaxis2 = dict(
        title = "Differential Stress [GPa]",
        overlaying = 'x',
        side = 'top',
        range = [0, 2]
    )
)
#fig.show(renderer="colab") #Uncomment to show graphical representation in Google Colab platform

In [122]:
# Exporting Archean lithospheric strength envelope to pdf file. Uncomment to write output to file. 
#fig.write_image("Archean_lithospheric_strength_envelope.pdf")